In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR

In [0]:
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
from google.colab import files

uploaded = files.upload()

Saving Hitter.csv to Hitter.csv


In [0]:
df=pd.read_csv("./Hitter.csv")
df=df.dropna() #eksik değerler siliniyor
dms=pd.get_dummies(df[['League', 'Division', 'NewLeague']]) #kategorik değişkenler dummy e çevrildi.One hot encoding yaklaşımı yapılmış oluyor.
y=df["Salary"]  #bağımlı değişken
X_=df.drop(['Salary','League','Division','NewLeague'], axis=1).astype('float64')
X=pd.concat([X_, dms[['League_N','Division_W','NewLeague_N']]], axis=1) #oluşturulan dummyler ve bir önceki veri seti bir araya getirilerek bağımsız değişkenlerin nihai hali elde edildi.
X_train, X_test, y_train, y_test=train_test_split(X,
                                                  y,
                                                  test_size=0.25,
                                                  random_state=42) #veri seti 25e 75 ayrılmış oldu. test train

In [5]:
!pip install lightgbm

In [0]:
from lightgbm import LGBMRegressor

In [0]:
lgbm=LGBMRegressor().fit(X_train, y_train)

In [12]:
lgbm

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [13]:
y_pred=lgbm.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

363.8712087611089

**Model Tuning**

In [0]:
lgbm_params={"learning_rate": [0.1, 0.01, 0.5, 1],
            "max_depth":[1,2,3,4,5,6,7,8,9,10],
            "n_estimators":[20,40,100,200,500,1000]} #buradaki değerleri öndeğerlerin yakınlarındaki değerlere göre veriyoruz. öndeğerler üstteki sonuçlar

In [17]:
lgbm_cv_model=GridSearchCV(lgbm, lgbm_params, cv=10, n_jobs=-1, verbose=2).fit(X_train, y_train)

Fitting 10 folds for each of 240 candidates, totalling 2400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 680 tasks      | elapsed:   30.9s
[Parallel(n_jobs=-1)]: Done 1492 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 2400 out of 2400 | elapsed:  1.9min finished


In [18]:
lgbm_cv_model.best_params_

{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 20}

In [0]:
lgbm_tuned=LGBMRegressor(learning_rate=0.1,
                                    max_depth=6,
                                    n_estimators=20).fit(X_train,y_train)

In [20]:
y_pred=lgbm_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

371.5044868943621